# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
J. Liu  ->  J. Liu  |  ['J. Liu']


Arxiv has 63 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.12439
extracting tarball to tmp_2501.12439...

 done.


Found 125 bibliographic references in tmp_2501.12439/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2501.12449
extracting tarball to tmp_2501.12449... done.


S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']


Found 62 bibliographic references in tmp_2501.12449/J1641_monitoring_Letter.bbl.
Retrieving document from  https://arxiv.org/e-print/2501.12760


extracting tarball to tmp_2501.12760...

 done.
  0: tmp_2501.12760/natbib.tex, 96 lines
  1: tmp_2501.12760/sample631.tex, 535 lines
  2: tmp_2501.12760/natnotes.tex, 332 lines
  3: tmp_2501.12760/aassymbols.tex, 579 lines


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2501.12760/aassymbols.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.12439-b31b1b.svg)](https://arxiv.org/abs/2501.12439) | **Introducing the AIDA-TNG project: galaxy formation in alternative dark matter models**  |
|| G. Despali, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-01-23*|
|*Comments*| *20 pages, 18 figures, submitted to A&A. Comments welcome, get in touch to know more about opportunities to use the runs - this https URL*|
|**Abstract**|            We introduce the AIDA-TNG project, a suite of cosmological magnetohydrodynamic simulations that simultaneously model galaxy formation and different variations of the underlying dark matter model. We consider the standard cold dark matter model and five variations, including three warm dark matter scenarios and two self-interacting models with constant or velocity-dependent cross-section. In each model, we simulate two cosmological boxes of 51.7 and 110.7 Mpc on a side, with the same initial conditions as TNG50 and TNG100, and combine the variations in the physics of dark matter with the fiducial IllustrisTNG galaxy formation model. The AIDA-TNG runs are thus ideal for studying the simultaneous effect of baryons and alternative dark matter models on observable properties of galaxies and large-scale structures. We resolve haloes in the range between $10^{8}$ and $4\times10^{14}\,$M$_{\odot}$ and scales down to the nominal resolution of 570 pc in the highest resolution runs. This work presents the first results on statistical quantities such as the halo mass function and the matter power spectrum; we quantify the modification in the number of haloes and the power on scales smaller than 1 Mpc, due to the combination of baryonic and dark matter physics. Despite being calibrated on cold dark matter, we find that the TNG galaxy formation model can produce a realistic galaxy population in all scenarios. The stellar and gas mass fraction, stellar mass function, black hole mass as a function of stellar mass and star formation rate density are very similar in all dark matter models, with some deviations only in the most extreme warm dark matter model. Finally, we also quantify changes in halo structure due to warm and self-interacting dark matter, which appear in the density profiles, concentration-mass relation and galaxy sizes.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.12449-b31b1b.svg)](https://arxiv.org/abs/2501.12449) | **Intervening nuclear obscuration changing the X-ray look of the $z\approx6$ QSO CFHQS J164121+375520**  |
|| F. Vito, et al. -- incl., <mark>S. Belladitta</mark> |
|*Appeared on*| *2025-01-23*|
|*Comments*| *Accepted for publication on A&A Letters*|
|**Abstract**|            X-ray observations of the optically selected $z=6.025$ QSO CFHQS J164121+375520 (hereafter J1641) revealed that its flux dropped by a factor $\gtrsim7$ from 2018, when it was a bright and soft X-ray source, to 2021. Such a strong variability amplitude has not been observed before among $z>6$ QSOs, and the underlying physical mechanism was unclear. We carried out a new X-ray and rest-frame UV monitoring campaign of J1641 over 2022-2024. We detected J1641 with Chandra in the 2-7 keV band, while no significant emission is detected at softer X-ray energies, making J1641 an X-ray changing look QSO at $z>6$. Comparing with the 2018 epoch, the 0.5-2 keV flux dropped dramatically by a factor $>20$. We ascribe this behaviour to intervening, and still ongoing, obscuration by Compton-thick gas intercepting our line of sight between 2018 and 2021. The screening material could be an inner disk or a failed nuclear wind that increased their thickness. Another possibility is that we have witnessed an occultation event due to dust-free clouds located at sub-pc/pc scales, similar to those recently invoked to explain the remarkable X-ray weakness of AGN discovered by JWST. These interpretations are also consistent with the lack of strong variations of the QSO rest-frame UV lightcurve over the same period. Future monitoring of J1641 and the possible discovery of other X-ray changing look QSOs at $z>6$ will provide us with precious information about the physics of rapid supermassive black-hole growth at high redshift.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.12760-b31b1b.svg)](https://arxiv.org/abs/2501.12760) | **The Impact of Bar-induced Non-Circular Motions on the Measurement of Galactic Rotation Curves**  |
|| <mark>J. Liu</mark>, Z. Li, J. Shen |
|*Appeared on*| *2025-01-23*|
|*Comments*| *19 pages, 12 figures, ApJ accepted*|
|**Abstract**|            We study the impact of bar-induced non-circular motions on the derivation of galactic rotation curves (RCs) using hydrodynamic simulations and observational data from the PHANGS-ALMA survey. We confirm that non-circular motions induced by a bar can significantly bias RCs derived from the conventional tilted-ring method, consistent with previous findings. The shape of the derived RC depends on the position angle difference ($\Delta \phi$) between the major axes of the bar and the disk in the face-on plane. For $\left|\Delta \phi\right|\lesssim40^\circ$, non-circular motions produce a bar-induced "dip" feature (rise-drop-rise pattern) in the derived RC, which shows higher velocities near the nuclear ring and lower velocities in the bar region compared to the true RC (${\mathrm{RC_{true}}}$). We demonstrate convincingly that such dip features are very common in the PHANGS-ALMA barred galaxies sample. Hydrodynamical simulations reveal that the "dip" feature is caused by the perpendicular orientation of the gas flows in the nuclear ring and the bar; at low $\left|\Delta \phi\right|$ streamlines in the nuclear ring tend to enhance $V_\mathrm{los}$, while those in the bar tend to suppress $V_\mathrm{los}$. We use a simple {\misaell} model to qualitatively explain the general trend of RCs from the tilted-ring method (${\mathrm{RC_{tilted}}}$) and the discrepancy between ${\mathrm{RC_{tilted}}}$ and ${\mathrm{RC_{true}}}$. Furthermore, we propose a straightforward method to implement a first-order correction to the RC derived from the tilted-ring method. Our study is the first to systematically discuss the bar-induced "dip" feature in the RCs of barred galaxies combining both simulations and observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.12439/./figures/profiles_dm3.png', 'tmp_2501.12439/./figures/mass_func_dmo.png', 'tmp_2501.12439/./figures/mass_func_hydro.png', 'tmp_2501.12439/./figures/halo386.png']
copying  tmp_2501.12439/./figures/profiles_dm3.png to _build/html/
copying  tmp_2501.12439/./figures/mass_func_dmo.png to _build/html/
copying  tmp_2501.12439/./figures/mass_func_hydro.png to _build/html/
copying  tmp_2501.12439/./figures/halo386.png to _build/html/
exported in  _build/html/2501.12439.md
    + _build/html/tmp_2501.12439/./figures/profiles_dm3.png
    + _build/html/tmp_2501.12439/./figures/mass_func_dmo.png
    + _build/html/tmp_2501.12439/./figures/mass_func_hydro.png
    + _build/html/tmp_2501.12439/./figures/halo386.png
found figures ['tmp_2501.12449/./J1641_UV_Xrayflux_lightcurve_NH_resize.png', 'tmp_2501.12449/./Xray_images_square.png']
copying  tmp_2501.12449/./J1641_UV_Xrayflux_lightcurve_NH_resize.png to _build/html/
copying  tmp_2501.12449/./Xray_images_square.png to _bu

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# Introducing the AIDA-TNG project:\\galaxy formation in alternative dark matter models

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.12439-b31b1b.svg)](https://arxiv.org/abs/2501.12439)<mark>Appeared on: 2025-01-23</mark> -  _20 pages, 18 figures, submitted to A&A. Comments welcome, get in touch to know more about opportunities to use the runs - this https URL_

</div>
<div id="authors">

G. Despali, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** We introduce the AIDA-TNG project, a suite of cosmological magnetohydrodynamic simulations that simultaneously model galaxy formation and different variations of the underlying dark matter model. We consider the standard cold dark matter model and five variations, including three warm dark matter scenarios and two self-interacting models with constant or velocity-dependent cross-section. In each model, we simulate two cosmological boxes of 51.7 and 110.7 Mpc on a side, with the same initial conditions as TNG50 and TNG100, and combine the variations in the physics of dark matter with the fiducial IllustrisTNG galaxy formation model. The AIDA-TNG runs are thus ideal for studying the simultaneous effect of baryons and alternative dark matter models on observable properties of galaxies and large-scale structures. We resolve haloes in the range between $10^{8}$ and $4\times10^{14} $ M $_{\odot}$ and scales down to the nominal resolution of 570 pc in the highest resolution runs. This work presents the first results on statistical quantities such as the halo mass function and the matter power spectrum; we quantify the modification in the number of haloes and the power on scales smaller than 1 Mpc, due to the combination of baryonic and dark matter physics. Despite being calibrated on cold dark matter, we find that the TNG galaxy formation model can produce a realistic galaxy population in all scenarios. The stellar and gas mass fraction, stellar mass function, black hole mass as a function of stellar mass and star formation rate density are very similar in all dark matter models, with some deviations only in the most extreme warm dark matter model. Finally, we also quantify changes in halo structure due to warm and self-interacting dark matter, which appear in the density profiles, concentration-mass relation and galaxy sizes.

</div>

<div id="div_fig1">

<img src="tmp_2501.12439/./figures/profiles_dm3.png" alt="Fig11" width="100%"/>

**Figure 11. -** Dark matter density profiles at $z=0$ when considering five halo mass bins (left to right). We calculate the mean dark matter profile in bins of $\Delta \log(M_{\rm vir})=0.2$ dex around the mean value, both in the full-physics (solid) and dark (dashed) runs. The profiles are calculated in logarithmically spaced spherical shells from 1 kpc to the halo virial radius. In order to capture both a good statistics at high masses and the highest resolution, we average profiles from both the 50/A and 100/A runs for all models -- except WDM1, where 50/B is employed. The small panels show, $(i)$ the ratio of each mean profile to its dark counterpart, and $(ii)$ to the CDM version. We observe an opposite trend in mass in the WDM and SIDM models: the latter create larger density cores at high masses and cuspy profiles at $M_{\rm vir}\leq 10^{12} {\rm M}_{\odot}$, the former instead lowers the central density at low masses, i.e. halo masses below the half-mode mass $M_{\rm hm}$. In all panels, the gray bands indicate 2.3$\epsilon_{\rm DM}$ for both runs, which is often used as the minimum distance for reliable measurements. (*fig:prof1*)

</div>
<div id="div_fig2">

<img src="tmp_2501.12439/./figures/mass_func_dmo.png" alt="Fig8.1" width="50%"/><img src="tmp_2501.12439/./figures/mass_func_hydro.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Halo mass function at $z=0$ in the DMO (left) and FP (right) runs of the four simulation sets listed in Table \ref{table:1}. For each set, we only plot haloes with masses $M_{\rm vir}\geq100 m_{\rm DM}$, corresponding to increasingly smaller low-mass limits as indicated by the labels in the top-left panel. The smaller panels show the ratio of each model to CDM. The dashed black line gives the theoretical prediction calculated by [Despali, Giocoli and Angulo (2016)]() -- which agrees very well with the mass function measured in the DMO runs, while the dotted lines are calculated by applying to it the correction derived by [Lovell, Frenk and Eke (2014)](). In the right panel, we also plot the CDM DMO mass function (black dashed line) to highlight the suppression generated by baryons at the low-mass end, compared to the DMO case. For the WDM runs, the transition between dashed and solid lines marks the limiting mass $M_{\rm lim}$ below which the runs are affected by artificial fragmentation. (*fig:massf*)

</div>
<div id="div_fig3">

<img src="tmp_2501.12439/./figures/halo386.png" alt="Fig5" width="100%"/>

**Figure 5. -** Visualisation of the dark matter (top), gas (middle) and stellar (bottom) projected mass distributions in a Milky-Way mass halo ($M_{\rm vir}=6.9\times10^{12} {\rm M}_{\odot}$ in CDM at $z=0$) from the 100/A runs: from left to right, CDM, SIDM1, vSIDM and WDM3. We zoom onto the central parts of the halo, $\sim0.5 r_{\rm vir}$ in the top and middle panels, and $\sim0.25 r_{\rm vir}$ in the stellar distribution. In the latter, we also mark the distance corresponding to twice the stellar half-mass radius. We can observe a decrease in the number of substructures in WDM3, as well as a clear difference in the gas dynamics and the shape of the stellar distribution.
               (*fig:visual1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.12439"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\chandra}{\textit{Chandra}\xspace}$
$\newcommand{\xmm}{\textit{XMM-Newton}\xspace}$
$\newcommand{\alma}{\textit{ALMA}\xspace}$
$\newcommand{\ang}{\mathrm{Å}\xspace}$
$\newcommand{\um}{~\mathrm{\mu m}\xspace}$
$\newcommand{\daox}{\mbox{\Delta\alpha_{ox}}}$
$\newcommand{\aox}{\mbox{\alpha_{\mathrm{ox}}}}$
$\newcommand{\LUV}{\mbox{L_{\mathrm{UV}}}}$
$\newcommand{\zsdss}{\mbox{z_{\mathrm{SDSS}}}}$</div>



<div id="title">

# Intervening nuclear obscuration changing the X-ray look of the $z\approx6$ QSO CFHQS J164121+375520

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.12449-b31b1b.svg)](https://arxiv.org/abs/2501.12449)<mark>Appeared on: 2025-01-23</mark> -  _Accepted for publication on A&A Letters_

</div>
<div id="authors">

F. Vito, et al. -- incl., <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** X-ray observations of the optically selected $z=6.025$ QSO CFHQS J164121+375520 (hereafter J1641) revealed that its flux dropped by a factor $\gtrsim7$ from 2018, when it was a bright and soft X-ray source, to 2021. Such a strong variability amplitude has not been observed before among $z>6$ QSOs, and the underlying physical mechanism was unclear. We carried out a new X-ray and rest-frame UV monitoring campaign of J1641 over 2022--2024. We detected J1641 with $\chandra$ in the 2--7 keV band, while no significant emission is detected at softer X-ray energies, making J1641 an X-ray changing look QSO at $z>6$ . Comparing with the 2018 epoch, the 0.5--2 keV flux dropped dramatically by a factor $>20$ . We ascribe this behaviour to intervening, and still ongoing, obscuration by  Compton-thick gas intercepting our line of sight between 2018 and 2021. The screening material could be an inner disk or a failed nuclear wind that increased their thickness. Another possibility is that we have witnessed an occultation event due to dust-free clouds located at sub-pc/pc scales, similar to those recently invoked to explain the remarkable X-ray weakness of AGN discovered by JWST. These interpretations are also consistent with the lack of strong variations of the QSO rest-frame UV lightcurve over the same period.	Future monitoring of J1641 and the possible discovery of other X-ray changing look QSOs at $z>6$ will provide us with precious information about the physics of rapid supermassive black-hole growth at high redshift.

</div>

<div id="div_fig1">

<img src="tmp_2501.12449/./J1641_UV_Xrayflux_lightcurve_NH_resize.png" alt="Fig2" width="100%"/>

**Figure 2. -** X-ray lightcurve of J1641 as a function of observation time: red circles and squares are the soft-band and hard-band observed fluxes, respectively. The most recent red points refer to the stacked $\chandra$ observations taken in 2022, 2023, and 2024, and are plotted at the average observing date (Tab. \ref{Tab_Xray_cts}), with horizontal error bars encompassing the times of the three stacked observations.
  The soft-band flux of J1641 dropped by a factor $>20$, while the QSO is still detected in the hard band, with at most a modest dimming. Black points are the $z$-band magnitudes of J1641 (\citetalias{Vito22} and Tab. \ref{Tab_UV}), and show no significant variation over the period covered by the X-ray observations.   (*Fig_lightcurve*)

</div>
<div id="div_fig2">

<img src="tmp_2501.12449/./Xray_images_square.png" alt="Fig1" width="100%"/>

**Figure 1. -** _Chandra_ images ($20^{\prime\prime}\times20^{\prime\prime}$) of J1641. The left and right columns display the 2018 epoch and the stacked image of the new $\chandra$ monitoring program (2022-2024), respectively, and the associated ACIS-S exposure time. Soft-band  and hard-band  images are shown in the top and bottom rows, respectively.The $R=2$\arcsec$$ circles are the apertures used for X-ray photometry.
	 (*Fig_Xray_images*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.12449"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

314  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
